In [ ]:
import sys
!python3 -m  pip install  happybase
!python3 -m pip install plotly
!python3 -m pip install shap
!python3 -m pip install "notebook>=5.3" "ipywidgets>=7.2"
!python3 -m pip install catboost
!python3 -m pip install xgboost

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns;sns.set()
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import matplotlib.pyplot as plt
from matplotlib import cm
from pandas import read_csv, set_option
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer,RobustScaler
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier,CatBoostRegressor
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.feature_selection import SelectKBest,f_regression
from xgboost import plot_importance,XGBClassifier,XGBRegressor
from sklearn.model_selection import learning_curve
from sklearn.pipeline import Pipeline
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import preprocessing
from datetime import datetime 
import shap
import happybase as hb


from sklearn.decomposition import PCA
from sklearn.decomposition import SparsePCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import MiniBatchDictionaryLearning
from sklearn.decomposition import FastICA
from sklearn.manifold import Isomap
from sklearn.manifold import MDS
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import TSNE
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection

# for dirname, _, filenames in os.walk('/kaggle/'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
        
import time
import warnings
warnings.filterwarnings('ignore')
sns.set(style='whitegrid')
%matplotlib inline

# time series cross validation
# https://hub.packtpub.com/cross-validation-strategies-for-time-series-forecasting-tutorial/

''' FUNCTIONS '''

# One plot type
def plot_line(ldf,lst,title='',sec_id=None,size=[350,1000]):
    
    # sec_id - list of [False,False,True] values of when to activate supblots; same length as lst
    
    if(sec_id is not None):
        fig = make_subplots(specs=[[{"secondary_y": True}]])
    else:
        fig = go.Figure()
        
    if(len(lst) is not 1):
        ii=-1
        for i in lst:
            ii+=1
            if(sec_id is not None):
                fig.add_trace(go.Scatter(x=ldf.index, y=ldf[lst[ii]],mode='lines',name=lst[ii],line=dict(width=2.0)),secondary_y=sec_id[ii])
            else:
                fig.add_trace(go.Scatter(x=ldf.index, y=ldf[lst[ii]],mode='lines',name=lst[ii],line=dict(width=2.0)))
    else:
        fig.add_trace(go.Scatter(x=ldf.index, y=ldf[lst[0]],mode='lines',name=lst[0],line=dict(width=2.0)))

    fig.update_layout(height=size[0],width=size[1],template='plotly_white',title=title,
                          margin=dict(l=50,r=80,t=50,b=40));fig.show()
    
# plot n verticle subplots
def plot_vsubplots(ldf,lst,title='',nplots=None,lw_id=None,size=[400,1000]):

    # lw_id list of line widths if added
        
    assert(nplots is not None) 
    fig = make_subplots(rows=nplots,shared_xaxes=True)
    ii=-1
    for i in lst:
        ii+=1
        fig.add_trace(go.Scatter(x=ldf.index,y=ldf[lst[ii]], mode='lines',name=lst[ii],line=dict(width=lw_id[ii])), row=ii+1, col=1) 

    fig.update_layout(height=size[0],width=size[1],template='plotly_white',title=title,
                          margin=dict(l=50,r=80,t=50,b=40));fig.show()
    
colours = ['tab:blue','tab:red','tab:green']
def plot_line2(ldf,lst,title=''):
    
    ii=-1
    plt.figure(figsize=(14,5))
    for i in lst:
        ii+=1
        ax = ldf[lst[ii]].plot(color=colours[ii],label=lst[ii],lw=1.5)
    plt.title(title)
    plt.legend();plt.show()
    
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Plot Correlation to Target Variable only
def corrMat(df,target='demand',figsize=(9,0.5),ret_id=False):
    
    corr_mat = df.corr().round(2);shape = corr_mat.shape[0]
    corr_mat = corr_mat.transpose()
    corr = corr_mat.loc[:, df.columns == target].transpose().copy()
    
    if(ret_id is False):
        f, ax = plt.subplots(figsize=figsize)
        sns.heatmap(corr,vmin=-0.3,vmax=0.3,center=0, 
                     cmap=cmap,square=False,lw=2,annot=True,cbar=False)
        plt.title(f'Feature Correlation to {target}')
    
    if(ret_id):
        return corr
    
def bar_plot(x, y,palette_len,title='Missing Values (%)', xlim = None, ylim = None, 
             xticklabels = None, yticklabels = None,xlabel = None, ylabel = None, 
             figsize = (10,4),axis_grid = 'y'):
        
    cmap = sns.color_palette("plasma")
    fig, ax = plt.subplots(figsize = figsize)
    plt.title(title,size = 15, fontweight = 'bold')

    for i in ['top', 'right', 'bottom', 'left']:
        ax.spines[i].set_color('black')
    
    ax.spines['top'].set_visible(True);ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False);ax.spines['left'].set_visible(False)

    sns.barplot(x = x, y = y, edgecolor = 'black', ax = ax,
                palette = cmap)
    ax.set_xlim(xlim);ax.set_ylim(ylim)    
    ax.set_xticklabels(xticklabels);ax.set_yticklabels(yticklabels)
    plt.xlabel(xlabel);plt.ylabel(ylabel)
    ax.grid(axis = axis_grid,ls='--',alpha = 0.9)
    plt.show()

# function to plot a two PCA Feature Plot using Pandas 
def scatterPlot(xDF, yDF, algoName):
    
    sns.set_style('whitegrid')
    fig, ax = plt.subplots()
    tempDF = pd.DataFrame(data=xDF.loc[:,0:1], index=xDF.index)
    tempDF = pd.concat((tempDF,yDF), axis=1, join="inner")
    tempDF.columns = ["Component 1","Component 2","Label"]
    g = sns.scatterplot(x="Component 1",y="Component 2",data=tempDF,hue="Label",
                        linewidth=0.5,alpha=0.5,s=15,edgecolor='k')
    plt.title(algoName);plt.legend()
    
    for i in ['top', 'right', 'bottom', 'left']:
        ax.spines[i].set_color('black')
    
    ax.spines['top'].set_visible(False);ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(False);ax.spines['left'].set_visible(False)
    ax.grid(axis = 'both',ls='--',alpha = 0.9)
    plt.show()
    
    
# reduce memory (@mfjwr1); distorts the data a little (but reduces by 60% memory)
def red_mem(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

# Split for TimeSeries
def TimeSeries_Split(ldf,
                     split_id=[None,None],
                     test_id=False,
                     cut_id=None):
    
    # Reduce the number of used data
    if(cut_id is not None):
        print('Reducing Input Data')
        
        if(type(cut_id) is int):
            ldf = ldf.iloc[-cut_id:]
        else:
            # input anything other than int
            print('Slicing based on period')
            ldf = ldf[data_period]
            
        t1 = ldf.index.max();t0 = ldf.index.min()
        print(f'Dataset Min.Index: {t0} | Max.Index: {t1}')
        
    if(split_id[0] is not None):
        # General Percentage Split (Non Shuffle requied for Time Series)
        train_df,pred_df = train_test_split(ldf,test_size=split_id[0],shuffle=False)
    elif(split_id[1] is not None):
        # specific time split 
        train_df = df.loc[:split_id[1]]; pred_df = df.loc[split_id[1]:] 
    else:
        print('Choose One Splitting Method Only')
        
#     y_train = train_df[feature]
#     X_train = train_df.loc[:, train_df.columns != feature]
#     if(test_id):
#         y_test = pred_df[feature]
#         X_test = pred_df.loc[:, pred_df.columns != feature]
        
    return train_df,pred_df # return 

In [ ]:
def  load_data_from_hbase(con:hb.Connection,tablename:str,row_start:str, row_stop):
    con.open()
    table = con.table(tablename)
    print(con.tables())
    candlesticks= [ data for key,data in table.scan(row_start=row_start.encode("utf-8"), row_stop=row_stop.encode("utf-8"))]
    df = pd.DataFrame(candlesticks).apply(lambda  x: x.apply(lambda  y:  float(y.decode("utf-8").replace('\'',''))))
    columns  = [c.decode("utf-8") for c in df.columns]
    df.columns =  columns
    df.rename(columns={"CANDLESTICKES:close": "Close", "CANDLESTICKES:open":"Open","CANDLESTICKES:high":"High","CANDLESTICKES:low":"Low","CANDLESTICKES:volume":"Volume_(Currency)", "CANDLESTICKES:close_time":"Timestamp"}, inplace=True)
    df['Timestamp'] = df['Timestamp'].apply(lambda x : int(x)//1000)
    df['date'] = df['Timestamp'].apply(lambda  x : datetime.fromtimestamp(x))
    df.set_index('date', inplace=True)
    con.close()
    return df

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#CDE10F'>PROBLEM DEFINITION</span></b></p></div>

- A major drawback of crypocurrency trading is the **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">volatility</mark>** of the market.
- The currency trades can occur 24/7 & tracking crypto position can be an impossible task to manage without automation.
- Automated Machine Learning trading algorithms can assist in managing this task, in order to predict the market's movement.
- We can use models to classify future movements into three categries: 

> <code>(1) The market will rise (take long position)</code>, <br>
> <code>(2) The market will fall (take short position)</code> <br>
> <code>(3) The market will move sideways (take no position)</code>.
    
    
- The problem of predicting a buy (<code>value=1</code>) or sell (<code>value=0</code>) signal for a **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">trading strategy</mark>** is defined in the
classification framework. 
- The buy or sell signal are decided on the basis of a comparison of short term vs. long
term price & is defined in <code>Section 2.2</code>
- Data harvesting (just data collection here) & <code>feature engineering</code> are relevant factors in time series model improvement. 
- It's interesting to investigate whether traditionally stock orientated feature engineering modifications are relevant to <code>digital assets</code>, and if so which ones
- Last but not least, **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">model generation efficiency</mark>** becomes much more significant when dealing with **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">High Frequency Tick Data</mark>** as each added feature can have a substatial impact on the turnaround time of a model, due to the amount of data & balancing model accuracy & model output turnaround time is definitely worth managing

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#CDE10F'>DEFINITION DU PROBLEME</span></b></p></div>

- Un inconvénient majeur du trading de cryptomonnaies est la **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">volatilité</mark>** du marché.
- Les échanges de devises peuvent avoir lieu 24h/24 et 7j/7 et le suivi de la position cryptographique peut être une tâche impossible à gérer sans automatisation.
- Les algorithmes de trading automatisés de Machine Learning peuvent aider à gérer cette tâche, afin de prédire le mouvement du marché.
- Nous pouvons utiliser des modèles pour classer les mouvements futurs en trois catégories : 

> <code>(1) Le marché va augmenter (prendre une position longue), <br>
> <code>(2) Le marché va baisser (prendre une position courte)</code> <br>
> <code>(3) Le marché évoluera latéralement (ne prendra pas de position).</code>.
    
- Le problème de la prédiction d'un signal d'achat (<code>valeur = 1</code>) ou de vente (<code>valeur = 0</code>) pour une **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">stratégie de trading</mark>** est défini dans le cadre de classification.
- Le signal d'achat ou de vente est décidé sur la base d'une comparaison des prix à court terme vs à long terme est défini dans la <code>section 2.2</code>.
-  La collecte de données (uniquement la collecte de données ici) et <code>l'ingénierie des fonctionnalités</code> sont des facteurs pertinents dans l'amélioration des modèles de séries chronologiques.
-  Il est intéressant de déterminer si les modifications d'ingénierie de fonctionnalités traditionnellement orientées sur les stocks sont pertinentes pour <code>les actifs numériques</code>, et si oui, lesquelles.
-  Enfin et surtout, **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">l'efficacité de la génération de modèles</mark>** devient beaucoup plus significative lorsqu'il s'agit de **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">données de ticks à haute fréquence</mark>**, car chaque fonctionnalité ajoutée peut avoir un impact substatiel sur le délai d'exécution d'un modèle, en raison de la quantité de données et de l'équilibrage de la précision du modèle et du délai de production du modèle. le temps vaut vraiment la peine d'être géré

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#CDE10F"><b><span style='color:#FFFFFF'>2 |</span></b> <b>LE JEU DE DONNÉES</b></div>

<div style=“color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;”><p style=“padding: 5px;color:white;text-align:left;”><b><span style=‘color:#CDE10F’>LECTURE DU JEU DE DONNÉES</span></b></p></div>

- Jeu de données actuel : Une extraction de chandeliers financières du bitcoin, la periode choisi est de octobre 2022 à octobre 2023 <code>à intervalle de 15 minutes<code>
- La caractéristique <code>timestamp</code> peut être analysée en un index de temps plus conventionnel en utilisant la bibliothèque <code>pytz</code>.
- Les <code>Caractéristiques de base</code> comprennent : <code>Open</code>, <code>High</code>, <code>Low</code>, <code>Close</code>, <code>Volume_(Currency)</code> de l’actif au quart d'heure
- Le jeu de données chargé contient un index de temps de début et de fin spécifiques, afin d’utiliser des modèles sur des données non vues, nous devons diviser le jeu de données et ne pas l’inspecter

In [ ]:
con = hb.Connection('hbase',9090)
df = load_data_from_hbase(con,"BINANCE",'BTCUSDT-15m#20221001', 'BTCUSDT-15m#20231001')
display(df.head(5))
display(df.tail(5))

In [ ]:
import datetime, pytz
#define a conversion function for the native timestamps in the csv file
def dateparse (time_in_secs):    
    return pytz.utc.localize(datetime.datetime.fromtimestamp(float(time_in_secs)))

# Data Periods used in Notebook
plot_period = slice('2023-01-19 14:59','2023-01-25 23:59') # Selectio Plot Period for visualisation only
data_period = slice('2022-10-01 14:59','2023-09-30 23:59') # Select Data Period for Analysis


In [ ]:
df.info()

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#CDE10F'>TRAVAILLER AVEC UN SOUS-ENSEMBLE DE DONNÉES</span></b></p></div>

- En raison de l’excès de données d’index disponibles pour nous, la formation des modèles peut devenir assez longue, surtout lorsqu’il s’agit de validation croisée.

In [ ]:
df_tr,df_te = TimeSeries_Split(df,split_id=[0.2,None], #  Train/Test Split (0.8/0.2)
                               cut_id=data_period)    

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#CDE10F"><b><span style='color:#FFFFFF'>3 |</span></b> <b>EXPLORER LES  DONNEES</b></div>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#CDE10F'>STATISTIQUE DESCRIPTIVE</span></b></p></div>

Étudions les statistiques des valeurs numériques de l'ensemble de données

In [ ]:
#set_option('precision',2)
df_tr.describe()

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#CDE10F'>VARIABLE CIBLE</span></b></p></div>

- Nous devons définir notre variable de prédiction <code>signal</code>, qui sera réalisée via <code>.rolling</code> & <code>.mean()</code> en utilisant la caractéristique <code>Close</code>.
- Une moyenne mobile à court terme (fenêtre), <code>SMA1</code> & une moyenne mobile à long terme (fenêtre), <code>SMA2</code> sont utilisées pour créer la variable cible, <code>signal</code>.
- La stratégie de trading est la suivante ; lorsque le <code>Short Term (SMA1)</code> > <code>Long Term (SMA2)</code>, la valeur du signal = 1 <code>(achat)</code>, sinon elle est fixée à 0 <code>(vente)</code>.
- La valeur de la moyenne mobile à <code>Short Term (SMA1)</code> & <code>Long Term (SMA2)</code> sont fixées à des <b>valeurs de fenêtre de 10 et 60</b> respectivement, toutes deux sont arbitraires, et peuvent affecter les résultats, idéalement une étude d’optimisation doit être réalisée pour trouver les valeurs optimales.

In [ ]:
def create_target(ldf,tr_id=False):
    ldf['SMA1'] = ldf['Close'].rolling(window=10, min_periods=1, center=False).mean() #  short simple moving average window
    ldf['SMA2'] = ldf['Close'].rolling(window=60, min_periods=1, center=False).mean() #  long simple moving average window
    ldf['signal'] = np.where(ldf['SMA1'] > ldf['SMA2'], 1.0, 0.0) # Create signals
    if(tr_id is not True):
        display(ldf['signal'].value_counts())
    
df_tr1 = df_tr.copy()  # Save the Baseline Model Dataframe [Training Set]
df_te1 = df_te.copy() # Save the Baseline Model Dataframe [Test Set]
create_target(df_tr1)  # Add target variable to Training Set 
create_target(df_te1,tr_id=True)  # Add target variable to Test Set

- Nous avons une distribution de <code>signal</code> de variable cible relativement uniforme (achat/vente) (13.8k/14.1k)
- Nous n'avons pas vraiment besoin de souligner les problèmes associés au <code>déséquilibre des classes</code> dans ce problème
- Des métriques simples telles que <code>accuracy,recall,precision</code> pourraient suffire comme métriques de <code>classification</code>, au lieu des courbes ROC et PR détaillées des modèles de classificateur.

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#CDE10F'>VISUALISATION DE SÉRIES TEMPORELLES DE JEUX DE DONNÉES</span></b></p></div>

Let's visualise the overall asset price history during the <code>training data</code> period & the associated <code>signal</code> as well.

In [ ]:
plot_vsubplots(df_tr1,['Close','signal'],
               title='Prix de cloture & signal des données d\'entrainement',
               nplots=2,
               lw_id=[2,0.4],
               size=[500,1000])

- Il est intéressant de noter la tendance générale à la hausse du prix <code>de cloture</code> des actifs, passant de 20k au début de cette période à 30k seulement en l'espace de quelques mois.
- Pas tout à fait facile à visualiser, le signal (ce que nous allons modéliser) est également tracé, et nous pouvons observer à quel point les MA de période plus courte et plus longue s'échangent au cours de cette seule courte période de temps.

In [ ]:
# Visualise Training Set Target Variable Related Features

lst_MAV = ['SMA1','SMA2','signal']
ldf = df_tr1.loc[plot_period,lst_MAV]
plot_line(ldf,lst_MAV,
          title='SM1, SMA2 & Signal created from Closing Price',
          sec_id=[False,False,True])  

Nous pouvons voir qu'il existe un certain nombre de périodes pendant lesquelles les valeurs de moyenne mobile les plus courtes et les plus longues s'échangent, même pour une période de <b>48 heures</b> seulement, au cours de laquelle le coût variait dans la plage de <code>21k:23k</code> au cours de la période observée, ce qui indique un actif très volatil.

In [ ]:
df_tr1=df_tr1.drop(['SMA1','SMA2','Timestamp'], axis=1)
df_te1=df_te1.drop(['SMA1','SMA2','Timestamp'], axis=1)   # replicate on test data

### <b><span style='color:#CDE10F'> 3.1 |</span> Caractéristiques du modèle de base</b>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#CDE10F'>CORRÉLATION DES CARACTÉRISTIQUES DE BASE</span></b></p></div>

Définissons le terme de "base" ; les caractéristiques qui sont disponibles pour être utilisées dans le jeu de données, c’est-à-dire open, close, etc.

Les valeurs de corrélation linéaire de nos caractéristiques actuelles <code>open</code>,<code>high</code>,<code>low</code>,<code>close</code>,<code>volumes</code> avec la variable cible sont très minimes.
Elles ne sont pas les plus idéales pour modéliser la cible, <code>signal</code>, et peuvent être améliorées, donc l’attention se déplace vers <code>le  feature engineering</code>

In [ ]:
corrMat(df_tr1,'signal',figsize=(7,0.5)) # Baseline Dataframe feature correlation to Signal

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#CDE10F'>INGÉNIERIE DES CARACTÉRISTIQUES</span></b></p></div>

-  Comme indiqué dans l’introduction, dans le problème actuel, nous nous concentrerons sur les <code>indicateurs techniques</code> dans le cadre de notre approche de<code>feature engineering</code> dans le but d’introduire des caractéristiques plus pertinentes dans la <code>matrice des caractéristiques</code>.
- Il est intéressant de savoir (dans le contexte d’un <code>actif numérique</code>), quelles caractéristiques ont un impact sur les performances du modèle, le cas échéant.
  
<b>Plus précisément :</b>

- <code>Moyenne mobile</code> : Une moyenne mobile donne une indication de la tendance du mouvement des prix en réduisant la quantité de bruit. <br>
- <code>Oscillateur stochastique %K et %D</code> : Un oscillateur stochastique est un indicateur de momentum comparant un prix de clôture particulier d’un titre à une gamme de ses prix sur une certaine période de temps. %K et %D sont des indicateurs lents et rapides. <br>
- <code>Indice de force relative (RSI)</code> : C’est un indicateur de momentum qui mesure l’ampleur des changements de prix récents pour évaluer les conditions de surachat ou de survente dans le prix d’une action ou d’un autre actif. Allant de [0,100]. <b>Actif -> 70 : actif considéré comme suracheté</b>. <b>Actif -> 30 : actif sous-vendu et sous-évalué.</b><br>
- <code>Taux de changement (ROC)</code>: C’est un oscillateur de momentum, qui mesure le pourcentage de changement entre le prix actuel et le prix passé n période. Les actifs avec des <b>valeurs ROC plus élevées</b> sont considérés comme plus susceptibles d’être surachetés et <b>ROC plus bas</b>; plus susceptibles d’être survendus.<br>
- <code>Momentum (MOM)</code> : C’est le taux d’accélération du prix ou du volume d’un titre ; la vitesse à laquelle le prix change. <br>
Peuvent tous être potentiellement utiles pour modéliser la variable cible, <code>signal</code>, avec bien sûr des degrés d’influence variables.

In [ ]:
df_tr2 = df_tr1.copy()  # Create duplicate dataframe & add features to it
df_te2 = df_tr2.copy()

In [ ]:
''' Technical Indicators '''

#Calculation of moving average
def ma(df, n):
    return pd.Series(df['Close'].rolling(n, min_periods=n).mean(), name='MA_' + str(n))

# exponentially weighted moving average 
def ema(df, n):
    return pd.Series(df['Close'].ewm(span=n,min_periods=n).mean(), name='EMA_' + str(n))

#Calculation of price momentum
def mom(df, n):     
    return pd.Series(df.diff(n), name='Momentum_' + str(n))  

# rate of change
def roc(df, n):  
    M = df.diff(n - 1) ; N = df.shift(n - 1)  
    return pd.Series(((M / N) * 100), name = 'ROC_' + str(n)) 

# relative strength index
def rsi(df, period):
    delta = df.diff().dropna()
    u = delta * 0; d = u.copy()
    u[delta > 0] = delta[delta > 0]; d[delta < 0] = -delta[delta < 0]
    u[u.index[period-1]] = np.mean( u[:period] ) #first value is sum of avg gains
    u = u.drop(u.index[:(period-1)])
    d[d.index[period-1]] = np.mean( d[:period] ) #first value is sum of avg losses
    d = d.drop(d.index[:(period-1)])
    rs = u.ewm(com=period-1, adjust=False).mean() / d.ewm(com=period-1, adjust=False).mean()
    return 100 - 100 / (1 + rs)

# stochastic oscillators slow & fast
def sto(close, low, high, n,id): 
    stok = ((close - low.rolling(n).min()) / (high.rolling(n).max() - low.rolling(n).min())) * 100
    if(id is 0):
        return stok
    else:
        return stok.rolling(3).mean()

In [ ]:
def tech_indi(ldf,tr_id=True):

    ''' Moving Average '''
    ldf['MA21'] = ma(ldf,10)
    ldf['MA63'] = ma(ldf, 30)
    ldf['MA252'] = ma(ldf, 200)
    lst_MA = ['MA21','MA63','MA252']

    ''' Exponentially Weighted Moving Average '''
    ldf['EMA10'] = ema(ldf, 10)
    ldf['EMA30'] = ema(ldf, 30)
    ldf['EMA200'] = ema(ldf, 200)
    lst_EMA = ['EMA10','EMA30','EMA200']

    ''' Momentum '''
    ldf['MOM10'] = mom(ldf['Close'], 10)
    ldf['MOM30'] = mom(ldf['Close'], 30)
    lst_MOM = ['MOM10','MOM30']

    ''' Relative Strength Index '''
    ldf['RSI10'] = rsi(ldf['Close'], 10)
    ldf['RSI30'] = rsi(ldf['Close'], 30)
    ldf['RSI200'] = rsi(ldf['Close'], 200)
    lst_RSI = ['RSI10','RSI30','RSI200']

    ''' Slow Stochastic Oscillators '''
    ldf['%K10'] = sto(ldf['Close'], ldf['Low'], ldf['High'],5,0)
    ldf['%K30'] = sto(ldf['Close'], ldf['Low'], ldf['High'],10,0)
    ldf['%K200'] = sto(ldf['Close'], ldf['Low'], ldf['High'], 20,0)
    lst_pK = ['%K10','%K30','%K200']

    ''' Fast Stochastic Oscillators '''
    ldf['%D10'] = sto(ldf['Close'], ldf['Low'], ldf['High'], 10,1)
    ldf['%D30'] = sto(ldf['Close'], ldf['Low'], ldf['High'], 30,1)
    ldf['%D200'] = sto(ldf['Close'], ldf['Low'], ldf['High'], 200,1)
    lst_pD = ['%D10','%D30','%D200']
    
    # Plot Training Data
    if(tr_id):
        plot_line(ldf.loc[plot_period,lst_MA],lst_MA,title='Moving Average (window=21,63,252)')
        plot_line(ldf.loc[plot_period,lst_EMA],lst_EMA,title='Exponential Moving Average (window=10,30,200)')
        plot_line(ldf.loc[plot_period,lst_MOM],lst_MOM,title='Momentum')
        plot_line(ldf.loc[plot_period,lst_RSI],lst_RSI,title='Relative Strength Index')
        plot_line(ldf.loc[plot_period,lst_pK],lst_pK,title='Stochastic Oscillators (slow)')
        plot_line(ldf.loc[plot_period,lst_pD],lst_pD,title='Stochastic Oscillators (Fast)')

In [ ]:
tech_indi(df_tr2) # add technical features to training set
tech_indi(df_te2,tr_id=False) # add technical features to test set

In [ ]:
# All the current features
df_tr2.columns

### <b><span style='color:#CDE10F'> 3.2 |</span> Mise à jour des caractéristiques (features) du modèle</b> 

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#CDE10F'>CORRÉLATION LINÉAIRE DES CARACTERISTIQUES MISE À JOUR</span></b></p></div>

- Avoir créé de nouvelles caractéristiques ; <code>MA</code>,<code>EMA</code>,<code>MOM</code>,<code>RSI</code>,<code>%K/%D</code>,
- étudions la corrélation linéaire de ces nouvelles caractéristiques avec la variable cible et comparons-les aux caractéristiques de l'<code>ensemble des caractéristiques de base</code>

In [ ]:
corrMat(df_tr2,'signal',figsize=(15,0.5))

- Nous pouvons voir le groupe de caractéristiques significativement plus linéairement corrélées qui ont été créées à la suite du <code>feature engineering</code>.
- Il est probable que les caractéristiques de l'ensemble de données <code>base</code> n'auront que peu ou pas d'impact sur la variation de la variable cible si elles sont utilisées dans la <code>matrice de caractéristiques</code>.
- D'un autre côté, les caractéristiques nouvellement créées ont une plage de valeurs corrélées raisonnablement large, et assez importante ; ne sont pas trop fortement corrélés à la variable cible, <code>signal</code>

In [ ]:
def drp_feat(ldf):
    ldf = ldf.drop(['High','Low','Open','Volume_(Currency)'], axis=1) # let's drop most of the original feature
    
drp_feat(df_tr2)
drp_feat(df_te2)

Après avoir appliqué des fonctions à notre <code>matrice de caractéristiques</code>, nous devons revérifier les données manquantes.

In [ ]:
NaN_values = (df_tr2.isnull().sum() / len(df_tr2) * 100).sort_values(ascending = False)
bar_plot(x = NaN_values,y = NaN_values.index,palette_len = NaN_values.index, 
         xlim = (0,1),xticklabels = range(0,10),yticklabels = NaN_values.index,
         figsize = (10,5), axis_grid = 'x')

In [ ]:
df_tr2 = df_tr2.dropna() 
df_te2 = df_te2.dropna()
df_tr2.info()

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow : Hidden;background-color:#CDE10F"><b><span style='color:#FFFFFF'>4 |</span></b> <b>GÉNÉRATION DE MODÈLES</b></div>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text- align:left;"><b><span style='color:#CDE10F'>FONCTION D'ÉVALUATION</span></b></p></div>


Après avoir défini une <code>variable cible</code> et une <code>matrice de caractéristiques</code> claires, passons en revue ce que nous avons : <br><br>
- df_tr1/df_te1 : <code>Dataframe d'entraînement/test des caractéristiques de base associées à l'actif</code>
- df_tr2/df_te2 : <code>DataFrame de données d'entrainement/test des caractéristiques nouvellement créées lors de l'étape de  feature engineering </code>

Et nous pouvons commencer à créer des modèles pour prédire la variable cible <code>signal</code> (directivité du marché), en utilisant la fonction d'évaluation ci-dessous.

<b>La fonction d'évaluation est (cachée ci-dessous) :</b><br><br>
L'objectif de la fonction d'évaluation est d'évaluer les performances du modèle sur différentes approches de répartition et d'évaluation des données.

<b>(1)</b> La fonction prend dans un <code>dataframe</code> qui contient à la fois la <code>matrice de caractéristiques, X</code> et la <code>variable cible, y</code> . <br>
<b>(2)</b> Les données sont divisées en deux parties ; <code>train_df</code> & <code>eval_df</code> <br>
<b>(3)</b> Une évaluation par <code>cross-validation</code> en 5-Folds de la dataframe de données importée est évaluée pour obtenir une image de la performance du modèle sur les données d'entraînement (petits et gros morceaux )<br>
<b>(4)</b> Un partage bidirectionnel standard (sans brassage de données) est effectué et entraîné sur <code>X_train/y_train</code> et <code>X_eval/y_eval</code>


In [ ]:
models = []
# Lightweight Models 
models.append(('LDA', LinearDiscriminantAnalysis()))  # Unsupervised Model 
models.append(('KNN', KNeighborsClassifier()))  # Unsupervised Model
models.append(('TREE', DecisionTreeClassifier())) # Supervised Model
models.append(('NB', GaussianNB())) # Unsupervised Model

# More Advanced Models
models.append(('GBM', GradientBoostingClassifier(n_estimators=25)))
models.append(('XGB',XGBClassifier(n_estimators=25,eval_metric='logloss')))
models.append(('CAT',CatBoostClassifier(silent=True,
                                        n_estimators=25)))
models.append(('RF', RandomForestClassifier(n_estimators=25)))

In [ ]:
# eval_id (T/F): [CV,Train,Test,all]

def modelEval(ldf,feature='signal',split_id=[None,None],eval_id=[True,True,True,True],
              n_fold=5,scoring='accuracy',plot_id=[False,True],cv_yrange=None,hm_vvals=[0.5,1.0,0.75]):
    
    print('Evaluation Function')
    print(f'Cross Validation Activated, n_splits : {n_fold}, scoring metric: {scoring}')
    if(eval_id[2]):
        if(split_id[0] is not None):
            print(f'Train/Evaluation Set Spit Activated: {split_id[0]}')
        if(split_id[1] is not None):
            print(f'Train/Evaluation Set Split made at {split_id[1]}')
    
    ''' 1. Split Train/Evaluation <DataFrame> Set Split '''
    
    # split_id : Train/Test split [%,timestamp], whichever is not None
    # test_id : Evaluate trained model on test set only
    
    if(split_id[0] is not None):
        # General Percentage Split (Non Shuffle requied for Time Series)
        train_df,eval_df = train_test_split(ldf,test_size=split_id[0],shuffle=False)
    elif(split_id[1] is not None):
        # specific time split 
        train_df = df.loc[:split_id[1]]; eval_df = df.loc[split_id[1]:] 
    else:
        print('Choose One Splitting Method Only')
        
    ''' 2. Train/Test Feature Matrices + Target Variables Split'''
    
    y_train = train_df[feature]
    X_train = train_df.loc[:, train_df.columns != feature]
    y_eval = eval_df[feature]
    X_eval = eval_df.loc[:, eval_df.columns != feature]
    X_one = pd.concat([X_train,X_eval],axis=0)
    y_one = pd.concat([y_train,y_eval],axis=0)
    
    print('');print(f'Using Features: {X_train.columns}')
    print(f'Target Variable: {feature}');print('')
        
    ''' 3. Visualise Training/Test Data'''
    if(plot_id[0]):
        
        # plot the training data
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=train_df.index, y=train_df['signal'],mode='lines',name='Training Data', line={'width': 0.25}))
        fig.update_layout(height=300,width=800,template='plotly_white',title='Training Signal Visualisation',
                          margin=dict(l=50,r=80,t=50,b=40))
        
        # Plot the test data as well 
        if(eval_id[2]):
            fig.add_trace(go.Scatter(x=eval_df.index, y=eval_df['signal'],mode='lines',name='Test Data',line={'width': 0.25}))
            fig.update_layout(title='Training/Test Signal Visualisation')
        fig.show()
    
    ''' 4. Cross Validation, Training/Evaluation, one evaluation'''
    lst_res = []; names = []; lst_train = []; lst_eval = []; lst_one = []; lst_res_mean = []
    if(any(eval_id)):
        for name, model in models:  # cycle through models & evaluate either cv or train/test
            names.append(name)
            
            # Cross Validation Model on Training Se
            if(eval_id[0]):
                t0=time.time()
                kfold = KFold(n_splits=n_fold)
                cv_res = cross_val_score(model,X_train,y_train, cv=kfold, scoring=scoring)
                t1 = time.time()
                lst_res.append(cv_res)
                tt1 = t1-t0 # total time for n_fold cross evaluation
                
            # Evaluate Fit Model on Training Data
            t2 = time.time()
            if(eval_id[1]):
                t2 = time.time()
                res = model.fit(X_train,y_train)
                train_res = accuracy_score(res.predict(X_train),y_train); lst_train.append(train_res)
            if(eval_id[2]):
                if(eval_id[1] is False):  # If training hasn't been called yet
                    res = model.fit(X_train,y_train)
                eval_res = accuracy_score(res.predict(X_eval),y_eval); lst_eval.append(eval_res)
            t3 = time.time()
            tt2 = t3-t2 # total time for training/evaluation train/prediction
            
            # Evaluate model on entire dataset
            if(eval_id[3]):
                t4 = time.time()
                res = model.fit(X_one,y_one)
                one_res = accuracy_score(res.predict(X_one),y_one); lst_one.append(one_res)
                t5 = time.time()
                tt3 = t5-t4 # total time for training & evaluation on whole dataframe
            
            ''' [out] Verbal Outputs '''
            # Cross Validation / Training / Evaluation Model Evaluation / Section Times
            lst_res_mean.append(cv_res.mean())
            fn1 = cv_res.mean(); fn2 = cv_res.std();
            fn3 = train_res; fn4 = eval_res; fn5 = one_res
            print(f"{name} : {fn1:.3f}({fn2:.3f}) -> {tt1:.2f}s | {fn3:.3f} & {fn4:.3f} -> {tt2:.2f}s | {fn5:.3f} -> {tt3:.2}s")
      
    s0 = pd.Series(np.array(lst_res_mean),index=names)
    s1 = pd.Series(np.array(lst_train),index=names)
    s2 = pd.Series(np.array(lst_eval),index=names)
    s3 = pd.Series(np.array(lst_one),index=names)
    pdf = pd.concat([s0,s1,s2,s3],axis=1)
    pdf.columns = ['cv_average','train','test','all']
    s4 = pd.Series([tt1,tt2,tt3],index=['cv','train/test','all'])
        
    ''' 5. Visual Ouputs '''
    if(plot_id[1]): 
        
        sns.set(style="whitegrid")
        fig,ax = plt.subplots(1,2,figsize=(15,4))
        ax[0].set_title(f'{n_fold} Cross Validation Results')
        sns.boxplot(data=lst_res, ax=ax[0], orient="v",width=0.3)
        ax[0].set_xticklabels(names)
        sns.stripplot(data=lst_res,ax=ax[0], orient='v',color=".3",linewidth=1)
        ax[0].set_xticklabels(names)
        ax[0].xaxis.grid(True)
        ax[0].set(xlabel="")
        if(cv_yrange is not None):
            ax[0].set_ylim(cv_yrange)
        sns.despine(trim=True, left=True)
    
        sns.heatmap(pdf,vmin=hm_vvals[0],vmax=hm_vvals[1],center=hm_vvals[2],
                    ax=ax[1],square=False,lw=2,annot=True,fmt='.3f',cmap='Blues')
        ax[1].set_title('Accuracy Scores')
        plt.show()

### <b><span style='color:#CDE10F'> 4.1 |</span> ÉVALUATION DU MODÈLE DE CARACTÉRISTIQUES DE BASE</b>

- Les **<span style='color:#CDE10F'>Caractéristiques de base</span>** comprennent :
> Les 15 minutes de l’actif <code>open</code>,<code>high</code>,<code>low</code>,<code>close</code> & <code>Volume_(Currency)</code>
- Comme je l’ai découvert, dans les applications de séries temporelles, il n’est pas très courant d’utiliser des caractéristiques de base associées à un seul actif, mais voyons comment cela se passe de toute façon, traçons également les données d’entraînement pour avoir une idée visuelle de ce que nous essayons de modéliser.

In [ ]:
vmodelEval(df_tr1,split_id=[0.2,None],plot_id=[False,True])

- Nous pouvons voir que le **<span style='color:#CDE10F'>cross_val_score</span>** se situe aux alentours de **<span style='color:#CDE10F'>précision = 0.5</span>**, ce qui suggère que l’utilisation de **<span style='color:#CDE10F'>caractéristiques de base</span>** associées à un seul actif n’est pas tout à fait adaptée pour prédire avec précision la **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">directivité de l’actif</mark>**

- La plupart des modèles avaient tendance à avoir un **<span style='color:#CDE10F'>score d’entraînement</span>** plus élevé que le **<span style='color:#CDE10F'>score de validation croisée</span>**

- Il était intéressant de voir que le **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">DecisionTreeClassifier</mark>** & **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">RandomForest</mark>**, même avec très peu d’estimateurs sont capables d’atteindre des scores très élevés (même s’ils sont en sur-apprentissage)

- Cela suggère que les modèles basés sur des arbres pourraient être très utiles dans ce problème et **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">kNN</mark>** peut également être ajouté à la liste des modèles surajustés sur les données d’entraînement car ils ont tendance à avoir des scores de validation croisée plus faibles

Le **<span style='color:#CDE10F'>temps d’entraînement et d’évaluation</span>** est également assez important dans ce problème :

- N’ayant utilisé que (100k/4.5M), le coût même avec 7 caractéristiques est assez élevé pour les modèles plus avancés, (<code>esp.GBM & ANN</code>);
- Les modèles plus avancés ont dû être considérablement réduits pour réduire le temps d’entraînement à des niveaux comparables, donc il est souhaitable d’optimiser le processus de sélection des caractéristiques.
- **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">XGB</mark>** & **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">CATBoost</mark>**, surprenant étaient assez rapides pour un modèle assez avancé, indiquant qu’il est assez bien optimisé pour être utilisé directement sorti de la boîte.
- **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">RandomForest</mark>** étant un modèle similaire à XGB est beaucoup plus lent.


### <b><span style='color:#CDE10F'> 4.2 |</span> Évaluation du modèle de caractéristiques mises à jour</b>
Nous avons créé de nouvelles caractéristiques avec le <code>feature engineering</code> dans la <code>Section 2.5</code>, générant les caractéristiques mises à jour : dataframe <code>df_feat</code>, essayons à nouveau avec ces nouvelles caractéristiques.

In [ ]:
modelEval(df_tr2,
          split_id=[0.2,None],
          plot_id=[False,True],
          cv_yrange=(0.8,1.0),
          hm_vvals=[0.8,1.0,0.9])

- Nous pouvons voir une amélioration très significative des **<span style='color:#CDE10F'>scores de précision</span>**, par rapport au **<span style='color:#CDE10F'>modèle de base</span>**
- **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">LinearDiscriminantAnalysis()</mark>** fonctionne étonnamment bien, non seulement sur l’ensemble d’entraînement mais aussi en validation croisée, c’est aussi l’une des approches les plus rapides, ce qui en fait l’une des approches les plus efficaces pour les grands ensembles de données
- Parmi les modèles à score plus élevé **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">LDA</mark>**, ne sont pas étonnamment des modèles plus avancés, **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">GBM</mark>**,**<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">XGB</mark>**,**<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">CAT</mark>**,**<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">RF</mark>** également
- **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">kNN()</mark>** et **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">GaussianNB()</mark>** les modèles non supervisés ont légèrement moins bien performé, en comparaison avec les modèles d’apprentissage supervisé

# <div style="padding: 30px;color:white;margin:10;font-size:60%;text-align:left;display:fill;border-radius:10px;background-color:#FFFFFF;overflow:hidden;background-color:#CDE10F"><b><span style='color:#FFFFFF'>5 |</span></b> <b>OPTIMISATION DE L’EFFICACITÉ DU MODÈLE</b></div>

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#CDE10F'>UTILISATION DE LA RÉDUCTION DE DIMENSIONNALITÉ</span></b></p></div>

- Un gros problème rencontré lors de la tentative d’atteindre l’objectif dans le problème est le grand nombre de données tick (chaque minute) dans l’ensemble de données, ce qui augmente considérablement le coût d'entrainement et d’évaluation.
- Comme les **<span style='color:#CDE10F'>matrices de caractéristiques</span>** dépendent du nombre de caractéristiques et d’instances, une réduction d’une seule caractéristique inutile aurait un impact visible sur le coût de calcul.
- Il est donc de la plus haute importance de <b>réduire autant que possible les caractéristiques inutiles</b> au problème, équilibrant entre la précision du modèle et la vitesse d’entraînement/prédiction.

<b>Examinons deux approches que l’on pourrait prendre :</b>

- <b>(1) Réduction de dimensionnalité via l’évaluation de l’importance des caractéristiques (qui est un processus plus manuel) </b>

> Bien que cela soit plus un processus manuel puisque toutes les bibliothèques ne sont pas combinées, essayons de trouver un terrain d’entente entre toutes les approches et de les combiner en une seule fonction d’évaluation de l’<code>importance des caractéristiques</code> pour nous permettre d’identifier, d’évaluer et de supprimer les caractéristiques pour accélérer notre approche.

- <b>(2) Réduction de dimensionnalité à l’aide d’algorithmes d’apprentissage non supervisé (processus plus automatisé)</b>

> Une collection assez simple de puissants algorithmes de <code>réduction de dimension</code> est à notre disposition dans la bibliothèque <code>sklearn</code>, le seul problème auquel je peux penser est que ; expliquer ce que signifient les caractéristiques résultantes peut être un peu problématique.


### <b><span style='color:#CDE10F'> 5.1 |</span> Réduction de dimensionnalité via l’importance des caractéristiques</b>
- Nous pouvons examiner l’<b>Importance des caractéristiques</b> (FI) de certains modèles formés pour comprendre quelles caractéristiques et dans quelle mesure.
- Nous pouvons utiliser de telles fonctions minimalistes pour évaluer rapidement l’importance des caractéristiques en nous appuyant sur la <b>variation des approches</b> et les <b>bibliothèques optimisées</b>.
- Nous pouvons obtenir une <b>importance relative des caractéristiques</b> en utilisant différentes bibliothèques, la <b>fonction</b> <code>feature_importance</code> comprend :
> - **<span style=‘color:#CDE10F’>Corrélation linéaire</span>** avec la fonction abs().
> - **<span style=‘color:#CDE10F’>Valeurs SHAP</span>** du modèle de régression Catboost (n_est)
> - **<span style=‘color:#CDE10F’>Régresseur RandomForest</span>** (n_est)
> - **<span style=‘color:#CDE10F’>Régresseur XGBoost</span>** (n_est)
> - **<span style=‘color:#CDE10F’>Régresseur CatBoost</span>** (n_est)
> - **<span style=‘color:#CDE10F’>SelectKBest</span>** (k)

<div style="color:white;display:fill;border-radius:8px;font-size:100%; letter-spacing:1.0px;"><p style="padding: 5px;color:white;text-align:left;"><b><span style='color:#CDE10F'>AJUSTEMENTS SUPPLÉMENTAIRES</span></b></p></div>

- Les scores individuels sont combinés et mis à l’échelle en utilisant <code>MinMaxScaler()</code> & Plot.
- L’axe des y représente le score total (un score plus élevé est meilleur, max -> Nombre d’approches).
- L’axe des x représente les caractéristiques correspondantes du dataframe d’entrée.


In [ ]:
from sklearn.feature_selection import SelectKBest,f_regression
from xgboost import plot_importance,XGBRegressor
from catboost import CatBoostClassifier,CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
import shap
import seaborn as sns

cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Plot Correlation to Target Variable only
def corrMat(df,target='signal',figsize=(9,0.5),ret_id=False):
    
    corr_mat = df.corr().round(2)
    shape = corr_mat.shape[0]
    corr_mat = corr_mat.transpose()
    corr = corr_mat.loc[:, df.columns == target].transpose().copy()
    
    if(ret_id):
        return corr

''' Feature Importance '''
# Various Approaches for quick FI evaluation

def fi(ldf,target='signal',n_est=25,num_only=True,
       drop_id=None,target_cat=True,drop_na=False):
    
    # Select only numerical features
    if(num_only):
        ldf = ldf.select_dtypes(include=['float64','int64'])    
    
    # Drop all NaN
    if(drop_na):
        print(f'Before NaN drop: {ldf.shape}')
        ldf = ldf.dropna()
        print(f'After NaN dropped: {ldf.shape}')
    
    
    ldf = ldf.copy()
    # If target is categorical string variable
    if(target_cat):
        cats = ldf[target].unique()
        cats_id = [i for i in range(0,len(cats))]
        maps = dict(zip(cats,cats_id))    
        ldf[target] = ldf[target].map(maps)
    
    # If any features are desired to be droped 
    if(drop_id is not None):
        ldf = ldf.drop(drop_id,axis=1)

    # Input dataframe containing feature & target variable
    y = ldf[target]
    X = ldf.drop(target,axis=1)
    
#   CORRELATION
    imp = corrMat(ldf,target,figsize=(15,0.5),ret_id=True)
    del imp[target]
    s1 = imp.squeeze(axis=0);s1 = abs(s1)
    s1.name = 'CORR'
    
#   SHAP
    model = CatBoostRegressor(silent=True,n_estimators=n_est).fit(X,y)
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X)
    shap_sum = np.abs(shap_values).mean(axis=0)
    s2 = pd.Series(shap_sum,index=X.columns,name='CAT_SHAP').T
    
#   CATBOOST
    model = CatBoostRegressor(silent=True,n_estimators=n_est).fit(X,y)
    fit = model.fit(X,y)
    rf_fi = pd.DataFrame(model.feature_importances_,index=X.columns,
                                         columns=['CAT'])
    rf_fi.sort_values('CAT',ascending=False)
    s3 = rf_fi.T.squeeze(axis=0)
    
#   RANDOMFOREST
    model = RandomForestRegressor(n_est,random_state=0, n_jobs=-1)
    fit = model.fit(X,y)
    rf_fi = pd.DataFrame(model.feature_importances_,index=X.columns,
                                         columns=['RF'])
    rf_fi.sort_values('RF',ascending=False)
    s4 = rf_fi.T.squeeze(axis=0)

#   XGB 
    model=XGBRegressor(n_estimators=n_est,learning_rate=0.5,verbosity = 0)
    model.fit(X,y)
    data = model.feature_importances_
    s5 = pd.Series(data,index=X.columns,name='XGB').T

#   KBEST
    model = SelectKBest(k=5, score_func=f_regression)
    fit = model.fit(X,y)
    data = fit.scores_
    s6 = pd.Series(data,index=X.columns,name='KBEST')

    # Combine Scores
    df0 = pd.concat([s1,s2,s3,s4,s5,s6],axis=1)
    df0.rename(columns={'target':'lin corr'})

    # MinMax Scaler
    x = df0.values 
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled,index=df0.index,columns=df0.columns)
    df = df.rename_axis(f'<b>FI APPROACH</b>', axis=1)
    df = df.rename_axis('Feature', axis=0)
    
    pd.options.plotting.backend = "plotly"
    fig = df.plot(kind='bar',title='<b>SCALED FEATURE IMPORTANCE</b>',
                  color_discrete_sequence=px.colors.qualitative.T10)
    fig.update_layout(template='plotly_white',height=400,
                     font=dict(family='sans-serif',size=12),
                     margin=dict(l=60, r=40, t=50, b=10))
    fig.update_traces(width=0.25)
    fig.show()

In [ ]:
fi(df_tr2)

- On peut noter que pour beaucoup de caractéristiques, une petite valeur en magnitude de <code>corrélation</code> (valeur de Pearson) donne également de petites valeurs de score dans d'autres approches. De même, les caractéristiques avec une <code>corrélation</code> élevées ont tendance à avoir des scores élevés dans d'autres méthodes <code>d'importance des caractéristiques</code>, ce qui est assez intéressant. Il existe quelques exceptions, comme <code>%D10</code> et <code>MOM10</code>
- Bien que quelques caractéristiques montrent un léger désaccord en ce qui concerne l'importance des caractéristiques, dans l'ensemble, <b>une similarité des scores de caractéristiques peut être observée pour la plupart des approches</b>.
- Il est intéressant de noter des dizaines de cas de caractéristiques identiques (par exemple <code>MOM10</code>, <code>MOM30</code>) ; nous pouvons avoir une idée des nouvelles fonctionnalités potentielles que nous pourrions essayer (peut-être que <code>MOM20</code> aurait mieux fonctionné que <code>MOM10</code>)
- Nous pouvons observer de nombreuses caractéristiques qui ont une <b>valeur de score relatif très faible</b> pour la plupart des méthodes, et qui ont donc probablement peu ou pas d'impact, même si elles devaient être supprimées.
- Supprimer les caractéristiques potentiellement sans impact (soit environ 50 % d'entre elles) rendrait notre approche globale beaucoup plus efficace et nous permettrait de nous concentrer sur des recherches de grille <code>hyperparamètres</code> plus longues et plus approfondies qui, espérons-le, seront plus précises que l'un de nos modèles actuels


In [ ]:
#df_tr2_FI = df_tr2.drop(columns=['Open','High','Low','Close','Volume_(Currency)','MA63','EMA10','%K10', 'Timestamp'])
df_tr2_FI = df_tr2.drop(columns=['Open','High','Low','Close','Volume_(Currency)','MA21','MA252', 'MA63','EMA10','EMA200','%K10'])

In [ ]:
modelEval(df_tr2_FI,split_id=[0.2,None],plot_id=[False,True],cv_yrange=(0.8,1.0),hm_vvals=[0.8,1.0,0.9])

###  <b><span style='color:#CDE10F'> 5.2 |</span> Réduction de dimensionnalité à l’aide d’algorithmes d’apprentissage non supervisé</b>
Une approche alternative à la <mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">réduction des caractéristiques</mark> est l’utilisation de méthodes d’<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">apprentissage non supervisé</mark>.

- Nous devons sélectionner un algorithme (il est préférable de regarder plusieurs d’entre eux et de voir comment ils se comportent),
- Peut-être appliquer une certaine **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">mise à l’échelle</mark>** et simplement <code>fit_transform</code> pour obtenir la <code>matrice de caractéristiques</code> modifiée qui aura la dimension sélectionnée

La fonction suivante contient les algorithmes d’ **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">apprentissage non supervisé</mark>** sélectionnables suivants pour réaliser la réduction de dimensionnalité :

> - **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">PCA</mark>**, **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">Sparse PCA</mark>**, **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">Kernel PCA</mark>**, **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">Incremental PCA</mark>**, **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">Truncated SVD</mark>**
> - **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">Fast ICA</mark>**, **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">Gaussian Random Projection</mark>**, **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">Sparse Random Projection</mark>**
> - **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">IsoMap</mark>** (Manifold),**<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">MDS</mark>** (Manifold),**<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">TSNE</mark>** (Manifold)
> - **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">Locally Linear Embedding</mark>**, **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">Mini Batch Dictionary Learning</mark>**

- Tous ne sont pas réalisables sur Kaggle en raison de la mémoire computationnelle limitée (surtout les approches <b>Manifold</b>), même avec la fonction <code>red_mem</code> activée
- Il n’est pas trop rare d’utiliser des approches en plusieurs étapes pour les méthodes **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">manifold</mark>**, par exemple une étape **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">PCA</mark>** avant l’approche **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">manifold</mark>**
- Ici, nous utiliserons des approches qui nécessitent moins de ressources pour fonctionner


In [ ]:
def dimRed(ldf,feature='signal',split_id=[None,None],n_comp=5,plot_id=True,
           model_id='sparserandomprojection',scaler_id=[False,None]):
    
    # Given a dataframe, split feature/target variable
    X = ldf.copy()
    y = ldf[feature].copy()
    del X[feature]
    
    n_jobs = -1; rs = 32
    
    if(model_id is 'pca'):
        whiten = False
        model = PCA(n_components=n_comp,whiten=whiten,random_state=rs)
    if(model_id is 'sparsepca'):
        alpha = 1
        model = SparsePCA(n_components=n_comp,alpha=alpha,random_state=rs,n_jobs=n_jobs)
    elif(model_id is 'kernelpca'):
        kernel = 'rbf'; gamma = None
        model = KernelPCA(n_components=n_comp,kernel=kernel,gamma=gamma,n_jobs=n_jobs,random_state=rs)
    elif(model_id is 'incrementalpca'):
        batch_size = None
        model = IncrementalPCA(n_components=n_comp,batch_size=batch_size)
    elif(model_id is 'truncatedsvd'): 
        algorithm = 'randomized';n_iter = 5
        model = TruncatedSVD(n_components=n_comp,algorithm=algorithm,n_iter=n_iter,random_state=rs)
    elif(model_id is 'gaussianrandomprojection'):
        eps = 0.5
        model = GaussianRandomProjection(n_components=n_comp,eps=eps,random_state=rs)
    elif(model_id is 'sparserandomprojection'):
        density = 'auto'; eps = 0.5; dense_output = True
        model = SparseRandomProjection(n_components=n_comp,density=density, 
                                       eps=eps, dense_output=dense_output,random_state=rs)
    if(model_id is 'isomap'):
        n_neigh = 2
        model = Isomap(n_neighbors=n_neigh,n_components=n_comp, n_jobs=n_jobs)    
    elif(model_id is 'mds'):
        n_init = 1; max_iter = 50; metric = False
        model = MDS(n_components=n_comp,n_init=n_init,max_iter=max_iter,metric=True,
                    n_jobs=n_jobs, random_state=rs)
    elif(model_id is 'locallylinearembedding'):
        n_neigh = 10; method = 'modified'
        model = LocallyLinearEmbedding(n_neighbors=n_neigh,n_components=n_comp, method=method, \
                                    random_state=rs, n_jobs=n_jobs)
    elif(model_id is 'tsne'):
        learning_rate = 300; perplexity = 30; early_exaggeration = 12; init = 'random'
        model = TSNE(n_components=n_comp, learning_rate=learning_rate, \
                    perplexity=perplexity, early_exaggeration=early_exaggeration, \
                    init=init, random_state=rs)
    elif(model_id is 'minibatchdictionarylearning'):
        alpha = 1; batch_size = 200; n_iter = 25
        model = MiniBatchDictionaryLearning(n_components=n_comp,alpha=alpha,
                                            batch_size=batch_size,n_iter=n_iter,random_state=rs)
    elif(model_id is 'fastica'):
        algorithm = 'parallel'; whiten = 'unit-variance'; max_iter = 100
        model = FastICA(n_components=n_comp, algorithm=algorithm,whiten=whiten, 
                          max_iter=max_iter, random_state=rs)
    
    # Scaling 
    if(scaler_id[0]):
        
        opts = [StandardScaler(),RobustScaler(),MinMaxScaler(), Normalizer(norm='l2')]
        scaler = opts[scaler_id[1]].fit(X) 
        X_sca = pd.DataFrame(scaler.fit_transform(X),
                                       columns = X.columns,
                                       index = X.index) # summarize transformed data 
    
    # Unsupervised Dimension Reduction 
    if(scaler_id[0]):
        X_red = model.fit_transform(X_sca)
    else:
        X_red = model.fit_transform(X)
    X_red = pd.DataFrame(data=X_red, index=X.index)
    if(plot_id):
        scatterPlot(X_red, y,model_id)
    X_red[feature] = y
    
    return X_red # return new feature matrix

- Nous utilisons la réduction de dimensionnalité sur <code>df_tr2</code>, avant d'utiliser la fonction <code>modelEval</code> comme avant, en notant la précision et les temps d'exécution
- Pour la réduction de dimensionnalité, nous utiliserons ici **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">Fast ICA</mark>** avec différents **<mark style="background-color:#CDE10F;color:white;border-radius:5px;opacity:0.9">méthodes de mise à l'échelle des données</mark>**, vous pouvez essayer différentes combinaisons, le but étant pour obtenir la plus grande précision possible

In [ ]:
# Standard ICA (no scaling)
df_tr2_ICA = dimRed(df_tr2_FI,
                    split_id=[0.2,None],
                    model_id='fastica',
                    n_comp=5)

In [ ]:
modelEval(df_tr2_ICA,
          split_id=[0.2,None],
          plot_id=[False,True],
          cv_yrange=(0.8,1.0),
          hm_vvals=[0.8,1.0,0.9])

In [ ]:
'''StandardScaler ICA'''
df_tr2_ICA_sca0 = dimRed(df_tr2_FI,
                         split_id=[0.2,None],
                         model_id='fastica',
                         n_comp=5,
                         scaler_id=[True,0])

In [ ]:
modelEval(df_tr2_ICA_sca0,
          split_id=[0.2,None],
          plot_id=[False,True],
          cv_yrange=(0.8,1.0),
          hm_vvals=[0.8,1.0,0.9])

In [ ]:
'''RobustScaler ICA'''
df_tr2_ICA_sca1 = dimRed(df_tr2_FI,
                         split_id=[0.2,None],
                         model_id='fastica',
                         n_comp=5,
                         scaler_id=[True,1])

In [ ]:
modelEval(df_tr2_ICA_sca1,
          split_id=[0.2,None],
          plot_id=[False,True],
          cv_yrange=(0.8,1.0),
          hm_vvals=[0.8,1.0,0.9])

In [ ]:
'''MinMaxScaler ICA'''
df_tr2_ICA_sca2 = dimRed(df_tr2_FI,
                         split_id=[0.2,None],
                         model_id='fastica',
                         n_comp=5,
                         scaler_id=[True,2])

In [ ]:
modelEval(df_tr2_ICA_sca2,
          split_id=[0.2,None],
          plot_id=[False,True],
          cv_yrange=(0.8,1.0),
          hm_vvals=[0.8,1.0,0.9])

In [ ]:
'''Normaliser ICA'''
df_tr2_ICA_sca3 = dimRed(df_tr2_FI,
                         split_id=[0.2,None],
                         model_id='fastica',
                         n_comp=5,
                         scaler_id=[True,3])

In [ ]:
modelEval(df_tr2_ICA_sca3,
          split_id=[0.2,None],
          plot_id=[False,True],
          cv_yrange=(0.8,1.0),
          hm_vvals=[0.8,1.0,0.9])